# Factorización y primalidad 

In [1]:
from criptoMRG import *

### Tests de Primalidad

Utilizaremos la función <span style="color:blue">random</span> para generar aleatoriedad.

In [2]:
import random
from random import randint
random.random(), random.randint(1, 10**10)

(0.034010461584207397, 2616506131)

## Test 1: Pseudoprimos

- **Ejercicio 1.** Define una función <span style="color:red">psp(n)</span> con salida <span style="color:green">(b,true)</span> o <span style="color:green">(b,false)</span> y que haga los siguiente:
    1. Elige una base $b$ al azar.
    2. Comprueba si $gcd(b,n)=1$. 
        - Si es falso la función termina con la salida: 
            - <span style="color:green"> print $gcd(b,n)$ es divisor de $n$</span>.
            - (b,false)
        - Si es cierto comprueba si $b^{(n-1)}\equiv 1 \mbox{ mod } n$, 
            - Si es falso la salida es <span style="color:green">(b,false)</span>.
            - Si es cierto la salida es <span style="color:green">(b,true)</span>. 

- **Ejecicio 2.** Define una función <span style="color:red">psp(n,k)</span> que realice hasta k veces la función <span style="color:red">psp(n)</span>. 
    - Si en algún momento n falla el test la función termina con <span style="color:green">(b,false)</span>,
    con b la base para la cual n falla el test. 
    - Si pasa el test para las k bases elegidas la salida será:
        - <span style="color:green"> print "es posible que n sea primo"</span>
        - <span style="color:green"> retun (b,true)</span>
   puedes poner la última base o las k bases que has ido eligiendo.
    

In [5]:
# Pseudoprimalidad
# Comprueba que n es pseudoprimo en base b
def isPsp(n,b):
    return gcd(b,n) == 1 and pow(b,n-1,n) == 1


# Test simple utilizando una sola base aleatoria
def pspTest(n):
    
    # Elige una base aleatoria
    b = random.randint(2,n-1)
    if not isPsp(n,b):
        return b, False

    return b, True


# Test de pseudoprimalidad completo
def psp(n,k=1):
    
    # Prueba el test de primalidad simple sobre un conjunto de k bases
    # elegidas aleatoriamente.
    bases = []
    for _ in xrange(k):
        
        # Si alguna falla, sabemos inmediatamente que no es primo.
        b,result = pspTest(n)
        if result == False: return (b,False)
        bases.append(b)
    
    # Si todas las bases pasan el test, es posible que sea primo.
    print "Es posible que {} sea primo".format(n)
    return bases,True

In [6]:
psp(29)
psp(523,12)

Es posible que 29 sea primo
Es posible que 523 sea primo


([505, 472, 445, 299, 470, 480, 328, 308, 200, 237, 62, 224], True)

## Test 2: Pseudo primos de Euler. Test de Slovay-Strassen

### Símbolos de Jacobi y Legendre

#### Símbolo de Legendre
El símbolo de Legendre se define como:

$$
\left(\frac{a}{p}\right) = \left\{
\begin{array}{rl}
0 & \text{si } a \equiv 0 \pmod{p},\\
1 & \text{si } a \not\equiv 0\pmod{p} \text{ y es residuo cuadrático: } \;a\equiv x^2\pmod{p},\\
-1 & \text{si } a \not\equiv 0\pmod{p} \text{ y no es residuo cuadrático.}
\end{array}
\right.
$$

#### Símbolo de Jacobi
El símbolo de Jacobi se define como:

$$
\left(\frac{a}{n}\right) = \left(\frac{a}{p_1}\right)^{\alpha_1}\left(\frac{a}{p_2}\right)^{\alpha_2}\cdots \left(\frac{a}{p_k}\right)^{\alpha_k}
$$

Siendo $n=p_1^{\alpha_1}p_2^{\alpha_2}\cdots p_k^{\alpha_k}$ la factorización de $n$.

#### Ley de reciprocidad cuadrática y propiedades
Tanto el símbolo de Jacobi como el símbolo de Legendre cumplen que:

$$
\left(\frac{m}{n}\right)\left(\frac{n}{m}\right) = (-1)^{\tfrac{m-1}{2}\cdot\tfrac{n-1}{2}}
$$

Usaremos además las siguientes propiedades para su cálculo:

  - $\left(\frac{m}{n}\right) = \left(\frac{m+kn}{n}\right)$, para reducir el "numerador".
  - $\left(\frac{2}{n}\right) = (-1)^{\frac{n^2-1}{8}}$
  - $\left(\frac{1}{n}\right) = 1$, caso base de recursión.

In [7]:
# Utiliza funciones de la librería sympy
from sympy import gcd
from sympy.ntheory.factor_ import factorint

def jacobiSym(m,n):
    """Calcula el símbolo de Jacobi de dos enteros. El segundo de ellos
    no debe ser par."""
    
    # Trabaja en módulo n
    m = m % n
    
    # Caso base, no son coprimos
    if gcd(m,n) != 1:
        return 0
    
    # Caso base, 1 es un residuo cuadrático 
    if m == 1:
        return 1
    
    # Caso recursivo, m divisible entre 2
    if m % 2 == 0:
        return pow(-1, (n**2-1)/8) * jacobiSym(m/2, n)
    
    # Caso recursivo, ley de reciprocidad cuadrática
    return pow(-1, ((m-1)*(n-1))/4) * jacobiSym(n,m)

In [8]:
from sympy.ntheory import jacobi_symbol
jacobi_symbol(7,45), jacobiSym(7,45)

(-1, -1)

### Pseudoprimalidad de Euler

Recordamos que $n$ es pseudo primo de Euler respecto a la base $b$ si:

- $n$ es impar,
- $gcd(b,n)=1$ y
- $b^{\frac{n-1}{2}}\equiv jacobi(b,n) \mbox{ mod } n$.

- **Ejercicio 3.** Define la función <span style="color:red">epsp(n)</span> que:
    1. Si $n$ es par tiene como salida:
        - <span style="color:green">print n es par</span>
        - <span style="color:green">return (2,false)</span>
    2. Elige una base $b$ al azar.
    3. Comprueba si $gcd(b,n)=1$ 
       - Si es falso la función termina con:
         - <span style="color:green">print gcd(b,n) es divisor de n</span> 
         - <span style="color:green"> return (b,false)</span> 
       - Si es cierto comprueba si $b^{\frac{n-1}{2}} \equiv jacobi(b,n) \mbox{ mod } n$, 
          - si es falso la salida es <span style="color:green">(b,false)</span>,
          - si es cierto la salida es <span style="color:green">(b,true)</span>. 

- **Ejercicio 4.** Define una función <span style="color:red">epsp(n,k)</span> que realice hasta k veces el test 2. 
    - Si en algún momento $n$ falla el test la función termina con <span style="color:green">(b,false)</span>,
      con $b$ la base para la cual $n$ no pasa el test. 
    - Si $n$ pasa el test para las $k$ bases elegidas la salida será:
        - <span style="color:green"> print es posible que n sea primo</span>
        - <span style="color:green"> return (b,true)</span>, puedes poner las última base o las k bases.

In [9]:
# Pseudoprimalidad de Euler
# Definición de pseudoprimos de Euler.
def isEpsp(n,b):
    """Comprueba la pseudoprimalidad de Euler de los 
    números pares.
    """
    
    # Comprueba coprimalidad
    if gcd(b,n) != 1:
        print "{} es divisor de {}".format(gcd(b,n), n)
        return b,False
    
    # Test de Euler
    return pow(b,(n-1)/2,n) == jacobi_symbol(b,n)


# Test simple de pseudoprimalidad con una base aleatoria
# sólo para números impares.
def epspSimpleTest(n):
    # Elige una base aleatoria
    b = random.randint(2,n-1)
    if not isEpsp(n,b):
        return b, False

    return b, True


# Test de pseudoprimalidad de Euler
def epsp(n,k=1):
    # Eliminamos el caso de un número par
    if n % 2 == 0:
        print "{} es par".format(n)
        return 2,False
    
    # Prueba el test de primalidad simple sobre un conjunto de k bases
    # elegidas aleatoriamente.
    bases = []
    for _ in xrange(k):
        
        # Si alguna falla, sabemos inmediatamente que no es primo.
        b,result = epspSimpleTest(n)
        if result == False: return (b,False)
        bases.append(b)
    
    # Si todas las bases pasan el test, es posible que sea primo.
    print "Es posible que {} sea primo".format(n)
    return bases,True

In [10]:
epsp(29)
epsp(561,12)

Es posible que 29 sea primo


(28, False)

## Test 3. Fuertemente pseudoprimos. Test de Miller-Rabin

Recordar que $n$ es ***fuertemente pseudoprimo*** respecto a la base $b$ si: 
   - $n$ es impar, 
   - $gcd(b,n)=1$ y 
   - Si escribimos $n=2^s t$, con $t$ impar, entonces:
       - $b^t \equiv 1 \mbox{ mod } n$ o bien,
       - existe $i=0,1,\ldots,s-1$ tal que $b^{t * 2^i}\equiv -1 \mbox{ mod } n$.

- **Ejercicio 5.** Define una función <span style="color:red">mpot(p,m)</span> que calcule la mayor potencia de $p$ que divide a $m$. No deben usarse funciones que factoricen m.

In [11]:
def mpot(p, m):
    """Calcula el mayor exponente con el que p divide a m.
    La entrada debe ser un número positivo.
    """
    # Se comenta que en la práctica 10 será necesario que
    # esta función acepte números negativos.

    exp = 0
    while (m % p == 0):
        exp = exp+1
        m = m/p

    return exp

- **Ejercicio 6.**  Define una función <span style="color:red">fpsp(n)</span> que:
    - **1.** Si $n$ es par tiene como salida <span style="color:green"> print n es par, return (2,false)</span>.
    - **2.** En otro caso:
        - **2.1.** Calcule $s$ y $t$ tales que $n-1 = 2^s t$, con $t$ impar.
        - **2.2.** Elije una base $b$ al azar.
        - **2.3.** Comprueba si $gcd(b,n)=1$ 
             - ***2.3.1.*** Si es falso la función termina con <span style="color:green"> print gcd(b,n) es divisor de n, return (b false)</span>. 
             - ***2.3.2.*** Si es cierto comprueba si $b^t= \pm  1\mbox{ mod } n$: 
                  - ***2.3.2.1.*** Si es cierto la salida es <span style="color:green">(b,true)</span>, 
                  - ***2.3.2.2.*** Si es falso comprueba:
                      - **2.3.2.2.1.** Si existe un $i=1,\ldots,s-1$ tal que $b^{t*2^i}\equiv  -1 \mbox{ mod } n$ la salida es <span style="color:green">(i,b,true)</span> (sacar i es opcional).
                      - **2.3.2.2.2.** Si no existe tal $i$ la salida es <span style="color:green">(b, false)</span> . 

- **Ejercicio 7.**  Define una función <span style="color:red">fpsp(n,k)</span>  que aplique hasta $k$ veces el test 3. 
    - Si en algún momento $n$ no pasa el test la función termina con <span style="color:green">(b,false)</span>.
    - Si $n$ pasa el test para las $k$ bases elegidas la salida será <span style="color:green">print es posible que n sea primo, (bases, true)</span>

In [12]:
# Test de pseudoprimalidad de Euler
def fpsp(n, k=1):
    
    # Comprueba el test de primalidad para una base y una división
    # de n en partes par e impar dadas
    def fpspTest(n,b,s,t):
        # Comprueba coprimalidad
        if gcd(b,n) != 1:
            print "{} es divisor de {}".format(gcd(b,n),n)
            return b, False

        # Comprueba la primera condición de pseudoprimalidad de Euler
        powb = pow(b,t,n)
        if powb == 1 or powb == -1: return b, True

        # Comprueba la segunda condición de pseudoprimalidad de Euler
        for i in xrange(1,s):
            if pow(b, t*2**i, n) == n-1:
                return b, True
            if pow(b, t*2**i, n) == 1:
                return b, False
        
        # Si falla la comprobación
        return b, False
    
    # Caso unidad
    if n == 1: return 1, False
    
    # Caso par
    if n % 2 == 0:
        print "{} es par".format(n)
        return 2,n == 2
    
    # Descompone al número en parte par e impar
    s = mpot(2, n-1)
    t = (n-1)/s
    
    # Elige una base al azar para cada paso
    bases = []
    for _ in xrange(k):
        b = randint(2,n-1)
        bases.append(b)
        b, test = fpspTest(n,b,s,t)
        if not test: return b, False
    
    # En caso de que haya pasado todos los tests
    return bases, True

fpsp(561)

3 es divisor de 561


(48, False)

## Comparaciones

- **Ejercicio 8.** Comprueba los tres tests, usando números grandes y algún número de Carmichael, y compáralos con el test de Sympy <span style="color:blue">isprime</span>  

In [13]:
psp(561,15), epsp(561,15)

((187, False), (434, False))

# Factorización

### Funciones auxiliares

Lista de funciones auxiliares que vamos a definir:
   - <span style="color:red">abmod(x,n)</span>,
   - <span style="color:red">mayorpot(p,n)</span>,
   - <span style="color:red">bnumer(b,base,n)</span>,
   - <span style="color:red">vec_alfa(b,base,n)</span>,
   - <span style="color:red">parp(lista)</span>,
   - <span style="color:red">ssuma(lista1,lista2)</span>,
   - <span style="color:red">aux(k,r)</span>,
   - <span style="color:red">suma(lista,k)</span>,

Paso a comentar cada función:

  - La función <span style="color:red">abmod(x,n)</span> tiene como salida $x\%n$ si este es menor o igual que $\frac{n}{2}$ y $x\%n -n$ en caso contrario.  
  
  - La función <span style="color:red">mayorpot(p,x)</span>
     - Si $p=-1$ tiene como salida $1$ si $x<0$ y $0$ en caso contrario.
     - Para cualquier otro $p$ (normalmente primo) tiene como salida el exponente de la mayor potencia de $p$ que divide a $x$. Para definir esta función no puedes factorizar x.
     
  - La función <span style="color:red">bnumer(b,base,n)</span> tiene como salida true si $b$ es un base-número relativo a $n$ y false en caso contrario.
  
  - La función <span style="color:red">vec_alfa(b,base,n)</span> comprueba que $b$ es un base-número y en este caso tiene como salida el vector alfa asociado a $b$. 
Este será una lista de longitud <span style="color:green">len(base)</span> cuyas coordenadas son los exponentes de los elementos de "base" en la descomposición en primos de abmod($b^2,n$) (ver teoría).

  - La función <span style="color:red">parp(lista)</span> vale true si todos los elementos en la lista son pares y false en otro caso.
  
  - La función <span style="color:red">ssuma(lista1,lista2)</span> comprueba que las listas tienen la misma longitud y en ese caso tiene como salida una nueva lista con la misma longitud de las dos listas y que en lugar $i$ tiene a lista1[i]+lista2[i].

In [2]:
from criptoMRG import abmod,mayorpot,bnumer,vec_alfa,parp,ssuma

vec_alfa(3,[2,3,5],100)

[0, 2, 0]

Nuestro siguiente objetivo es calcular todas las <span style="color:red">ssumas</span> de k elementos de una lista de listas, de la misma longitud, con $k$ menor o igual que la longitud de cualquier lista de la lista.

   - Para ello vamos a definir una función auxiliar <span style="color:red">aux(k,r)</span> cuya salida sea una lista con todas las listas posibles de la forma $[l_0,l_2,\ldots,l_{k-1}]$ con
$0\leq l_0 < l_2 <\ldots< l_{k-1} \leq r.$
   
   - La función  <span style="color:red">suma(lista,k)</span>: La variable "lista" tiene que ser una lista de listas, todas ellas de la misma longitud, primero comprueba que $k\leq $<span style="color:green">len(lista)</span> y luego da una lista con todas las 
<span style="color:red">ssumas</span> posibles de $k$ elementos de la "lista"

In [1]:
from criptoMRG import suma, aux

## El algoritmo de factorización Factor Base

- ### *Elección de la base. Voy a tener suerte*

La primera elección de la base es poco profesional, por ejemplo elegimos la base con el -1 y los primeros 30 primos.

** Ejercicio 9.-** Define la base
$$B=[-1,2,3,...,113],$$ 
que tiene al $-1$ y los primeros 30 primos

In [4]:
from sympy import prime
basetreinta = [-1] + [prime(i) for i in xrange(1,30)]

- ### Elección de la lista de $B$ números. Voy a seguir teniendo suerte

Vamos a elegir una lista de $B$-números con la esperanza de que sean suficientes para resolver la ecuación $x^2\equiv y^2\mod n$. 

El proceso será el siguiente:

  - Fijamos $k_{max}$ y también $i_{max}$. 
  - Construimos la lista 
 $$L_1=[n_k=\mbox{ floor(sqrt($k*n$)), para } k=1,2,3,4,\ldots,k_{max}].$$ 
  
  - Construimos la lista $$L_2=[ b_{ki}=n_k + i, \mbox{ para } n_k \in L_1 \mbox{ e } i=0,2,3,\ldots,i_{max}-1]$$
 
  - Seleccionamos de $L_2$ aquellos $b_{ki}$ que son B-números y formamos la lista BN con ellos.
 
Define una función <span style="color:red">bi(n,k,i,base)</span> que realice el proceso de selección anterior.

In [5]:
from math import floor,sqrt

def bi(n,k,i,base):
    # Crea las listas
    lista1 = [int(floor(sqrt(i*n))) for i in xrange(1,k+1)]
    lista2 = [m+i for i in xrange(i) for m in lista1]
    
    # Selecciona los B-números
    return filter(lambda b: bnumer(b, base, n), lista2)

- ### Algoritmo de resolución de la ecuación $x^2\equiv y^2 \mod n$

  - Imput n
  - output (t,s) con $t^2=s^2\mod n$

    - **paso 1.-** Elegimos una base $B$ y esperamos tener suerte.
    - **paso 2.-** Elegimos índices $k_{max}$ e $i_{max}$ y construimos la lista de $B$-números <span style="color:green">BN</span>= <span style="color:red">bi(n,$k_{max}$,$i_{max}$,base)</span> y volvemos a esperar tener suerte.
    - **paso 3.-** Construimos la lista <span style="color:green">alfavec</span> de alfa vectores correspondientes a los $B$-números en $BN$.
    - **paso 4.-** Inicializamos el índice $j=1$, construimos la lista <span style="color:green">sumaj</span>=<span style="color:red">suma(alfavec,j)</span> y nos quedamos con las sublista <span style="color:green">sumajpar</span>  de sumaj cuyos elementos satisfagan la condición <span style="color:red">parp</span>. Si <span style="color:green">sumajpar</span> es vacía tomamos $j=j+1$ y repetimos el proceso anterior.
    - **paso 5.-** Inicializamos $\alpha$ como el primer elemento de <span style="color:green">sumajpar</span>, tomamos $in_\alpha$ como el lugar que ocupa $\alpha$ en <span style="color:green">sumaj</span>. Esto nos permite calcular la lista de eles para $\alpha$,
<center> <span style="color:green">eles$\alpha$</span> = <span style="color:red">aux(len(B),j)[in_$\alpha$]</span> = $[l_0,l_2,\ldots,l_{j-1}]$</center>
además tenemos que $$\alpha=\alpha_{l_0}+\ldots +\alpha_{l_{j-1}}.$$
Si ponemos $\alpha_{l_i}=(e_{l_i}^0,\ldots,e_{l_i}^h)$ y $\alpha=(e_0,\ldots,e_h)$ entonces:
$$ e_r=\sum_{i=0}^{j-1} e_{l_i}^r$$
    - **paso 6.-** Calculamos:
    $$ t=\prod_{i=0}^{j-1} b_{l_i} \quad\mbox{y}\quad s=\prod_{r=0}^h p_r^{\frac{e_r}{2}}$$
    - **paso 7.-** Si $(t,s)$ es una solución no trivial return $(t,s)$. En caso contrario volvemos al paso 5 tomando $\alpha=$ siguiente de $\alpha$.
    - **Fin.-** Si todas las soluciones que hemos obtenido son triviales no hemos tenido suerte y cambiamos primero los índices $k_{max}$ e $i_{max}$ y si aún así no tenemos suerte cambiamos la base $B$.

** Ejercicio 10.-** Elige un entero $n$ positivo que empiece por los dígitos de tu DNI, asegúrate de que el $n$ que has elegido no es primo y tampoco par. Realiza el algoritmo anterior paso a paso.

A partir de la solución que has obtenido, da una factorización de $n$.

***Comprueba el resultado.***

In [11]:
from sympy import isprime

# Elegimos un número que sabemos que no es primo
n = 7714566997697
isprime(n)

# Paso 1: Elegimos una base y esperamos tener suerte
base = basetreinta

# Paso 2: Elegimos índices con los que probar
k = 100
i = 10
bn = bi(n,k,i,base)

# Paso 3: Construimos la lista de alfa vectores
alfavec = map(lambda b: vec_alfa(b,base,n), bn)

# Paso 4:
# Paso 4.1, probamos y nos quedamos con los que cumplan la
# condición parp, que es la lista vacía
suma1 = suma(alfavec,1)
filter(parp,suma1)
# Paso 4.2, probamos y nos quedamos con los que cumplan la
# condición parp, que ahora son todos
suma2 = suma(alfavec,2)
suma2filtered = filter(parp,suma2)

# Paso 5: Calculamos la lista de eles
alpha = suma2filtered[0]
inalpha = 0
elesalpha = aux(2, len(base))[inalpha]
elesalpha

(0, 1)

** Ejercicio (Avanzado) 11.-** Define una función 
<center><span style="color:red">soleq(n,h,k,i)</span></center>
que haga lo siguiente:
- Tome como Base Factor a la lista formada por $-1$ seguido de los $h$-primeros primos.
- Tome $k$ e $i$ como los índices $k_{max}$ e $i_{max}$ en el algoritmo anterior
- Tenga como salida:
   - print todas las soluciones son triviales, o bien
   - $(t,s)$,  una solución no trivial de la ecuación $x^2\equiv y^2 \mod n$.

Comprueba que tu función "funciona" utilizando el $n$ del ejercicio anterior y viendo que obtienes los mismos resultados.

Aplica la función <span style="color:red">soleq</span> a varios ejemplos y comprueba los resultados.

** Ejercicio (Avanzado) 12.-** Define una función 
<center><span style="color:red">fac(n,h)</span></center>
para factrorizar $n$ que utilice la función soleq. El parámetro $h$ indica el número de primos que van a formar parte de la base factor. He omitido los índices $k$ e $i$ pero puedes añadirlos si los necesitas.

- ### Elección de la base. Fracciones continuas

- La función <span style="color:green">continued_fraction_periodic(a,b,d)</span> calcula la fracción continua asociada a $\frac{a+\sqrt d}{b}$.

- La función <span style="color:green">continued_fraction_convergents(lista)</span> calcula los convergentes.

- La función <span style="color:green">fraction</span> aplicada a una fracción separa el numerador y el denominador.

In [45]:
from sympy import *

In [46]:
F=continued_fraction_periodic(0,1,9073) # (0+sqrt(9073))/1
print F

[95, [3, 1, 26, 2, 6, 1, 1, 3, 2, 1, 5, 3, 1, 7, 5, 1, 1, 1, 4, 4, 4, 1, 1, 1, 5, 7, 1, 3, 5, 1, 2, 3, 1, 1, 6, 2, 26, 1, 3, 190]]


In [47]:
L1=[F[0]]+F[1][:5]
print L1

[95, 3, 1, 26, 2, 6]


In [48]:
L2=continued_fraction_convergents(L1)

In [49]:
L3=list(L2)
print L3

[95, 286/3, 381/4, 10192/107, 20765/218, 134782/1415]


In [50]:
fraction(L3[1])

(286, 3)

### Algoritmo de elección de la Base factor y de los B-números

- **Paso 1.-** Calculamos la fracción continua asociada a $\sqrt n$
<center> F = <span style="color:green">continued_fraction_periodic(0,1,n).</span></center>
- **Paso 2.-** Formamos la lista
<center> $L_1 =$ <span style="color:green">F[0]+F[1].</span></center>
- **Paso 3.-** Elegimos $s\leq len(L_1)$, calculamos los convergentes 
<center> $L_2 = $<span style="color:green">continued_fraction_convergents($L_1$[:s]).</span></center>
- **Paso 4.-** Formamos la lista Pbs cuyos elementos son los numeradores de los elementos en $L_2$.
- **Paso 5.-** Para cada $b\in Pbs$ factorizamos <span style="color:green">abmod $(b^2,n)$.</span>
- **Paso 6.-** La base factor $B$ estará formada por $-1$ junto con los primos que:
   - aparecen en la factorización de <span style="color:green">abmod $(b^2,n)$</span> para al menos dos b's en Pbs, o bien
   - aparece con un exponente par en la factorización de <span style="color:green">abmod $(b^2,n)$</span> para exactamente un b.
-  **Paso 7.-** La lista de B-números estará formada por aquellos b's en Pbs que sean B-números para la base obtenida en el paso anterior.

** Ejercicio 12.-** Elige un entero $n$ positivo que empiece por los dígitos de tu DNI, asegúrate de que el $n$ que has elegido no es primo y que es distinto del que elegiste en el ejercicio 1, y realiza el algoritmo anterior paso a paso obteniendo una base $B$ y una lista de $B$-números $BN$.

** Ejercicio 13.-** Completa el ejercicio 3 resolviendo la ecuación $x^2\equiv y^2\mod n$ utilizando como base factor y lista de B-números los obtenidos en el ejercicio 3. 

** Ejercicio 14.-** Completa el ejercicio 4 factorizando el $n$ que has elegido.

** Ejercicio (Avanzado) 15.-** Automatiza los procesos de elección de base factor y resolución de la ecuación, para ello define una función  <span style="color:red">soleqfc(n,s)</span>.

** Ejercicio (Avanzado) 16.-** Define una función <span style="color:red">facfc(n)</span> que factorice $n$ utilizando la función soleqfc.